In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
data_class = pd.read_csv('classification.csv')

In [4]:
data_class

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Visakhapatnam,07-07-2016,35.33,84.31,16.95,44.35,34.22,12.06,0.84,5.07,25.24,4.87,9.37,4.74,81.0,Satisfactory
1,Visakhapatnam,08-07-2016,40.64,92.73,28.48,52.89,47.01,11.91,1.07,6.57,16.24,6.46,12.20,6.81,112.0,Moderate
2,Visakhapatnam,09-07-2016,33.25,77.65,23.32,45.37,39.49,12.77,1.00,8.79,18.38,5.51,9.55,5.84,81.0,Satisfactory
3,Visakhapatnam,10-07-2016,26.64,69.68,11.03,36.51,26.00,12.82,0.88,9.97,27.98,4.12,6.82,3.48,81.0,Satisfactory
4,Visakhapatnam,11-07-2016,23.92,70.52,13.89,39.01,29.35,12.46,0.89,9.72,24.73,3.87,7.32,3.62,79.0,Satisfactory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,Visakhapatnam,25-06-2020,13.45,58.54,2.30,21.60,13.09,12.27,0.41,8.19,29.38,1.28,5.64,0.92,77.0,Satisfactory
1152,Visakhapatnam,26-06-2020,7.63,32.27,5.91,23.27,17.19,11.15,0.46,6.87,19.90,1.45,5.37,1.45,47.0,Good
1153,Visakhapatnam,27-06-2020,15.02,50.94,7.68,25.06,19.54,12.47,0.47,8.55,23.30,2.24,12.07,0.73,41.0,Good
1154,Visakhapatnam,28-06-2020,24.38,74.09,3.42,26.06,16.53,11.99,0.52,12.72,30.14,0.74,2.21,0.38,70.0,Satisfactory


In [9]:
data_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156 entries, 0 to 1155
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        1156 non-null   object 
 1   Date        1156 non-null   object 
 2   PM2.5       1153 non-null   float64
 3   PM10        1154 non-null   float64
 4   NO          1155 non-null   float64
 5   NO2         1148 non-null   float64
 6   NOx         1156 non-null   float64
 7   NH3         1151 non-null   float64
 8   CO          1154 non-null   float64
 9   SO2         1153 non-null   float64
 10  O3          1155 non-null   float64
 11  Benzene     1156 non-null   float64
 12  Toluene     1156 non-null   float64
 13  Xylene      1156 non-null   float64
 14  AQI         1153 non-null   float64
 15  AQI_Bucket  1153 non-null   object 
dtypes: float64(13), object(3)
memory usage: 144.6+ KB


In [11]:
data_class.isna().sum()

City          0
Date          0
PM2.5         3
PM10          2
NO            1
NO2           8
NOx           0
NH3           5
CO            2
SO2           3
O3            1
Benzene       0
Toluene       0
Xylene        0
AQI           3
AQI_Bucket    3
dtype: int64

In [13]:
data_class.dropna(inplace=True)

In [15]:
data_class.isna().sum()

City          0
Date          0
PM2.5         0
PM10          0
NO            0
NO2           0
NOx           0
NH3           0
CO            0
SO2           0
O3            0
Benzene       0
Toluene       0
Xylene        0
AQI           0
AQI_Bucket    0
dtype: int64

In [17]:
data_class.AQI_Bucket.unique()

array(['Satisfactory', 'Moderate', 'Good', 'Poor', 'Very Poor'],
      dtype=object)

In [19]:
data_class.columns

Index(['City', 'Date', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2',
       'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket'],
      dtype='object')

In [21]:
x = data_class[['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2',
       'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI']]
y = data_class[['AQI_Bucket']]

In [23]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x,y.values.ravel(),test_size=0.2,random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

In [27]:
len(x), len(x_train), len(x_test), len(x_val)

(1130, 678, 226, 226)

In [29]:
len(y), len(y_train), len(y_test), len(y_val)

(1130, 678, 226, 226)

In [31]:
x_train

array([[-1.44060047, -1.55778261, -0.39455697, ..., -0.90358147,
        -0.79512252, -1.32596314],
       [ 0.0328896 ,  0.60210437, -0.16310946, ...,  0.69149675,
         0.0503287 ,  0.2540836 ],
       [-1.09635021, -1.39630915, -0.07978835, ..., -0.0223696 ,
        -0.86186867, -1.19883295],
       ...,
       [-1.30417275, -1.70838174, -0.75191192, ..., -0.80825807,
        -0.83220371, -1.32596314],
       [-0.558415  ,  0.42360132,  0.22849973, ...,  0.82283121,
         0.32472953, -0.47237467],
       [-0.88817423, -0.32898398, -0.10756205, ..., -1.42044612,
        -0.98794472, -0.45421321]])

In [33]:
#knn

In [35]:
knn = KNeighborsClassifier()
parameters = {'n_neighbors': range(1, 50)}

In [37]:
grid_search_knn = GridSearchCV(knn, parameters, cv=5)
grid_search_knn.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 50)})

In [38]:
grid_search_knn.best_estimator_

KNeighborsClassifier(n_neighbors=17)

In [39]:
knn.fit(x_train,y_train)
y_pred_knn=grid_search_knn.predict(x_test)

In [40]:
print(confusion_matrix(y_pred_knn, y_test))

[[ 1  0  0  3  0]
 [ 0 91  4 12  0]
 [ 0  0 11  0  3]
 [ 8 16  0 77  0]
 [ 0  0  0  0  0]]


In [41]:
#svc

In [42]:
svc = SVC()

In [43]:
parameters={'C':[1.0,10, 100, 1000, 10000],
            'kernel':['rbf','linear','poly'],
            'gamma':['scale',0.1,0.01,0.001]
           }

In [44]:
grid_search_svc = GridSearchCV(svc, parameters, verbose=3)
grid_search_svc.fit(x_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5] END ....C=1.0, gamma=scale, kernel=rbf;, score=0.860 total time=   0.0s
[CV 2/5] END ....C=1.0, gamma=scale, kernel=rbf;, score=0.846 total time=   0.0s
[CV 3/5] END ....C=1.0, gamma=scale, kernel=rbf;, score=0.890 total time=   0.0s
[CV 4/5] END ....C=1.0, gamma=scale, kernel=rbf;, score=0.889 total time=   0.0s
[CV 5/5] END ....C=1.0, gamma=scale, kernel=rbf;, score=0.859 total time=   0.0s
[CV 1/5] END .C=1.0, gamma=scale, kernel=linear;, score=0.956 total time=   0.0s
[CV 2/5] END .C=1.0, gamma=scale, kernel=linear;, score=0.934 total time=   0.0s
[CV 3/5] END .C=1.0, gamma=scale, kernel=linear;, score=0.934 total time=   0.0s
[CV 4/5] END .C=1.0, gamma=scale, kernel=linear;, score=0.941 total time=   0.0s
[CV 5/5] END .C=1.0, gamma=scale, kernel=linear;, score=0.911 total time=   0.0s
[CV 1/5] END ...C=1.0, gamma=scale, kernel=poly;, score=0.868 total time=   0.0s
[CV 2/5] END ...C=1.0, gamma=scale, kernel=poly

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1.0, 10, 100, 1000, 10000],
                         'gamma': ['scale', 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'linear', 'poly']},
             verbose=3)

In [59]:
grid_search_svc.best_estimator_

SVC(C=10000, kernel='linear')

In [61]:
grid_search_svc.best_params_

{'C': 10000, 'gamma': 'scale', 'kernel': 'linear'}

In [63]:
y_pred_svc = grid_search_svc.predict(x_test)

In [65]:
print(confusion_matrix(y_pred_svc, y_test))

[[  5   0   0   0   0]
 [  0 107   0   0   0]
 [  0   0  14   0   2]
 [  4   0   0  92   0]
 [  0   0   1   0   1]]


In [67]:
#naive bayes

In [69]:
nb = GaussianNB()
nb.fit(x_train,y_train)
y_pred_nb = nb.predict(x_test)

In [71]:
print(confusion_matrix(y_test, y_pred_nb))

[[ 8  0  0  1  0]
 [ 0 89  2 16  0]
 [ 0  0 14  0  1]
 [ 7  9  0 76  0]
 [ 0  0  1  0  2]]


In [73]:
#validation

In [75]:
y_val_knn = grid_search_knn.predict(x_val)
y_val_svc = grid_search_svc.predict(x_val)
y_val_nb = nb.predict(x_val)

In [77]:
#knn
print(classification_report(y_test,y_pred_knn))
print(classification_report(y_val,y_val_knn))

              precision    recall  f1-score   support

        Good       0.25      0.11      0.15         9
    Moderate       0.85      0.85      0.85       107
        Poor       0.79      0.73      0.76        15
Satisfactory       0.76      0.84      0.80        92
   Very Poor       0.00      0.00      0.00         3

    accuracy                           0.80       226
   macro avg       0.53      0.51      0.51       226
weighted avg       0.78      0.80      0.78       226

              precision    recall  f1-score   support

        Good       0.83      0.38      0.53        13
    Moderate       0.79      0.82      0.80       102
        Poor       0.50      0.50      0.50        12
Satisfactory       0.75      0.81      0.78        94
   Very Poor       0.00      0.00      0.00         5

    accuracy                           0.76       226
   macro avg       0.57      0.50      0.52       226
weighted avg       0.74      0.76      0.74       226



In [79]:
#svc
print(classification_report(y_test, y_pred_svc))
print(classification_report(y_val, y_val_svc))

              precision    recall  f1-score   support

        Good       1.00      0.56      0.71         9
    Moderate       1.00      1.00      1.00       107
        Poor       0.88      0.93      0.90        15
Satisfactory       0.96      1.00      0.98        92
   Very Poor       0.50      0.33      0.40         3

    accuracy                           0.97       226
   macro avg       0.87      0.76      0.80       226
weighted avg       0.97      0.97      0.97       226

              precision    recall  f1-score   support

        Good       0.92      0.85      0.88        13
    Moderate       0.99      0.95      0.97       102
        Poor       0.67      0.83      0.74        12
Satisfactory       0.98      0.99      0.98        94
   Very Poor       0.67      0.80      0.73         5

    accuracy                           0.95       226
   macro avg       0.84      0.88      0.86       226
weighted avg       0.96      0.95      0.95       226



In [81]:
#naive bayes
print(classification_report(y_test,y_pred_nb))
print(classification_report(y_val,y_val_nb))

              precision    recall  f1-score   support

        Good       0.53      0.89      0.67         9
    Moderate       0.91      0.83      0.87       107
        Poor       0.82      0.93      0.88        15
Satisfactory       0.82      0.83      0.82        92
   Very Poor       0.67      0.67      0.67         3

    accuracy                           0.84       226
   macro avg       0.75      0.83      0.78       226
weighted avg       0.85      0.84      0.84       226

              precision    recall  f1-score   support

        Good       0.87      1.00      0.93        13
    Moderate       0.89      0.78      0.83       102
        Poor       0.55      1.00      0.71        12
Satisfactory       0.85      0.87      0.86        94
   Very Poor       1.00      0.60      0.75         5

    accuracy                           0.84       226
   macro avg       0.83      0.85      0.82       226
weighted avg       0.86      0.84      0.84       226



In [83]:
#knn
print(confusion_matrix(y_val_knn, y_val))

[[ 5  0  0  1  0]
 [ 0 84  6 17  0]
 [ 0  1  6  0  5]
 [ 8 17  0 76  0]
 [ 0  0  0  0  0]]


In [85]:
#svc
print(confusion_matrix(y_val_svc, y_val))

[[11  0  0  1  0]
 [ 0 97  1  0  0]
 [ 0  4 10  0  1]
 [ 2  0  0 93  0]
 [ 0  1  1  0  4]]


In [87]:
#nb
print(confusion_matrix(y_val_nb, y_val))

[[13  0  0  2  0]
 [ 0 80  0 10  0]
 [ 0  8 12  0  2]
 [ 0 14  0 82  0]
 [ 0  0  0  0  3]]


In [89]:
data_class['AQI'].isna().sum

<bound method Series.sum of 0       False
1       False
2       False
3       False
4       False
        ...  
1151    False
1152    False
1153    False
1154    False
1155    False
Name: AQI, Length: 1130, dtype: bool>

In [91]:
#unseen data

In [93]:
unseen = pd.read_csv('classification unseen.csv')

In [95]:
unseen.head(5)

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Visakhapatnam,30-09-2024,35.5,84.5,17.1,44.5,34.5,12.2,0.90,5.2,25.5,4.9,9.5,4.8,NaN,Satisfactory
1,Visakhapatnam,29-09-2024,NaN,91.9,28.6,53.0,47.2,12.0,1.10,6.7,16.5,6.5,12.4,6.9,113.0,Moderate
2,Visakhapatnam,28-09-2024,33.4,NaN,23.5,NaN,39.7,12.9,NaN,8.9,18.5,5.6,9.7,5.9,82.0,Satisfactory
3,Visakhapatnam,27-09-2024,26.8,69.8,11.2,36.7,26.2,13.0,0.95,10.1,NaN,4.2,7.0,3.5,NaN,Satisfactory
4,Visakhapatnam,26-09-2024,24.1,NaN,14.0,39.2,29.5,12.6,0.95,9.9,25.0,3.9,7.5,3.7,80.0,Satisfactory


In [97]:
unseen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        6 non-null      object 
 1   Date        6 non-null      object 
 2   PM2.5       5 non-null      float64
 3   PM10        4 non-null      float64
 4   NO          6 non-null      float64
 5   NO2         4 non-null      float64
 6   NOx         6 non-null      float64
 7   NH3         6 non-null      float64
 8   CO          5 non-null      float64
 9   SO2         6 non-null      float64
 10  O3          5 non-null      float64
 11  Benzene     5 non-null      float64
 12  Toluene     5 non-null      float64
 13  Xylene      5 non-null      float64
 14  AQI         4 non-null      float64
 15  AQI_Bucket  6 non-null      object 
dtypes: float64(13), object(3)
memory usage: 2.5+ KB


In [99]:
unseen = unseen.dropna(how='all')

In [101]:
unseen

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Visakhapatnam,30-09-2024,35.50,84.50,17.10,44.5,34.50,12.2,0.90,5.20,25.50,4.9,9.5,4.8,NaN,Satisfactory
1,Visakhapatnam,29-09-2024,NaN,91.90,28.60,53.0,47.20,12.0,1.10,6.70,16.50,6.5,12.4,6.9,113.0,Moderate
2,Visakhapatnam,28-09-2024,33.40,NaN,23.50,NaN,39.70,12.9,NaN,8.90,18.50,5.6,9.7,5.9,82.0,Satisfactory
3,Visakhapatnam,27-09-2024,26.80,69.80,11.20,36.7,26.20,13.0,0.95,10.10,NaN,4.2,7.0,3.5,NaN,Satisfactory
4,Visakhapatnam,26-09-2024,24.10,NaN,14.00,39.2,29.50,12.6,0.95,9.90,25.00,3.9,7.5,3.7,80.0,Satisfactory
5,Visakhapatnam,25-09-2024,108.49,191.72,23.48,NaN,46.26,13.4,1.10,14.45,40.81,NaN,NaN,NaN,265.0,Poor


In [103]:
unseen.isna().sum()

City          0
Date          0
PM2.5         1
PM10          2
NO            0
NO2           2
NOx           0
NH3           0
CO            1
SO2           0
O3            1
Benzene       1
Toluene       1
Xylene        1
AQI           2
AQI_Bucket    0
dtype: int64

In [105]:
unseen_x = unseen[['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2',
       'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI']]
unseen_y = unseen[['AQI_Bucket']]

In [107]:
unseen_scaled_x = scaler.transform(unseen_x)

In [109]:
unseen_scaled_x = pd.DataFrame(unseen_scaled_x, columns=unseen_x.columns)

In [111]:
training_means = x_train.mean()
unseen_filled_x = unseen_scaled_x.fillna(training_means)

In [113]:
unseen_filled_x.isna().sum()

PM2.5      0
PM10       0
NO         0
NO2        0
NOx        0
NH3        0
CO         0
SO2        0
O3         0
Benzene    0
Toluene    0
Xylene     0
AQI        0
dtype: int64

In [115]:
unseen_filled_x

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI
0,-0.420574,-0.462554,0.371997,0.442075,0.311613,0.182089,0.129129,-0.908595,-0.428149,0.188315,-0.028724,0.588006,0.007670
1,0.007670,-0.310723,1.436656,0.965911,1.151574,0.144018,0.688191,-0.729782,-0.758548,0.968824,0.585582,1.366711,-0.072823
2,-0.494796,0.007670,0.964503,0.007670,0.655534,0.315341,0.007670,-0.467524,-0.685126,0.529788,0.013641,0.995899,-0.635828
3,-0.728066,-0.764162,-0.174219,-0.038622,-0.237338,0.334377,0.268894,-0.324474,0.007670,-0.153158,-0.558299,0.105950,0.007670
4,-0.823495,0.007670,0.085002,0.115447,-0.019081,0.258233,0.268894,-0.348315,-0.446505,-0.299503,-0.452384,0.180113,-0.672151
5,2.159183,1.737343,0.962651,0.007670,1.089404,0.410520,0.688191,0.194083,0.133897,0.007670,0.007670,0.007670,2.687719


In [117]:
y_unseen_svc = grid_search_svc.predict(unseen_filled_x)

In [121]:
print(classification_report(unseen_y,y_unseen_svc))

              precision    recall  f1-score   support

    Moderate       0.33      1.00      0.50         1
        Poor       1.00      1.00      1.00         1
Satisfactory       1.00      0.50      0.67         4

    accuracy                           0.67         6
   macro avg       0.78      0.83      0.72         6
weighted avg       0.89      0.67      0.69         6



In [129]:
print(confusion_matrix(y_unseen_svc, unseen_y))

[[1 0 2]
 [0 1 0]
 [0 0 2]]
